En este código voy a tratar de automatizar el analisis de los diametros de las fibras para ver su cambio con la profundidad y cuando se han tratado

He visto librerías interesantes como DiameterJ lo que pasa es que esta es para ImageJ y yo quiero algo de python para poder extender a todas las imágenes ya que sino tendría que ir una por una. Es por eso que he visto otras alternativas como scikit-image que es muy avanzada en morfología y medidas (pero como nuestras imágenes son muy complicadas voy a empezar usando opencv para realizar un analisis previo sencillo)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize, remove_small_objects
from scipy.ndimage import distance_transform_edt
from skimage import measure, filters, segmentation
import pandas as pd
import tqdm

def analyze_fiber_diameters(image_folder, output_csv="fiber_diameters.csv", min_diameter=3, max_diameter=100):
    """
    Analiza automáticamente diámetros de fibras en un conjunto de imágenes
    :param image_folder: Carpeta con imágenes (TIFF, PNG, JPG)
    :param output_csv: Archivo CSV de salida con resultados
    :param min_diameter: Diámetro mínimo aceptable (píxeles)
    :param max_diameter: Diámetro máximo aceptable (píxeles)
    :return: DataFrame con resultados y gráficos estadísticos
    """
    # Configuración
    results = []
    valid_extensions = ('.tif', '.tiff', '.png', '.jpg', '.jpeg')
    
    # Procesar cada imagen
    for filename in tqdm.tqdm(os.listdir(image_folder)):
        if not filename.lower().endswith(valid_extensions):
            continue
            
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            continue
            
        # Preprocesamiento avanzado
        denoised = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)
        equalized = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)).apply(denoised)
        blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
        
        # Binarización adaptativa
        binary = cv2.adaptiveThreshold(
            blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, 51, 7
        )
        
        # Limpieza morfológica
        kernel = np.ones((3,3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Eliminar objetos pequeños
        cleaned = remove_small_objects(cleaned.astype(bool), min_size=50)
        
        # Esqueletización y transformada de distancia
        skeleton = skeletonize(cleaned)
        dist_transform = distance_transform_edt(cleaned)
        skeleton_dist = dist_transform * skeleton
        
        # Calcular diámetros (2 * radio)
        diameters = 2 * skeleton_dist[skeleton_dist > 0]
        
        # Filtrar diámetros válidos
        valid_diameters = diameters[(diameters > min_diameter) & (diameters < max_diameter)]
        
        if len(valid_diameters) > 0:
            # Estadísticas
            stats = {
                'filename': filename,
                'mean_diameter': np.mean(valid_diameters),
                'median_diameter': np.median(valid_diameters),
                'std_diameter': np.std(valid_diameters),
                'min_diameter': np.min(valid_diameters),
                'max_diameter': np.max(valid_diameters),
                'fiber_count': len(valid_diameters)
            }
            results.append(stats)
    
    # Generar reporte
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    
    # Gráficos estadísticos
    if not df.empty:
        plt.figure(figsize=(12, 8))
        
        plt.subplot(221)
        df['mean_diameter'].plot(kind='hist', bins=20, alpha=0.7)
        plt.title('Distribución de Diámetros Promedio')
        plt.xlabel('Diámetro (px)')
        
        plt.subplot(222)
        plt.scatter(df.index, df['median_diameter'], c=df['fiber_count'], cmap='viridis')
        plt.colorbar(label='Número de Fibras')
        plt.title('Diámetro Mediano por Imagen')
        plt.ylabel('Diámetro (px)')
        
        plt.subplot(223)
        df.boxplot(column=['mean_diameter', 'median_diameter'])
        plt.title('Comparación de Métricas')
        
        plt.subplot(224)
        fiber_counts = df['fiber_count'].value_counts()
        plt.pie(fiber_counts, labels=fiber_counts.index, autopct='%1.1f%%', startangle=90)
        plt.title('Distribución de Conteo de Fibras')
        
        plt.tight_layout()
        plt.savefig('fiber_diameter_stats.png', dpi=300)
    
    return df

# Uso =============================================
if __name__ == "__main__":
    # Configurar estos parámetros según tus imágenes
    IMAGE_FOLDER = r"C:\\Users\\danwo\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au"
    RESULTS_CSV = "resultados_diametros.csv"
    
    df_results = analyze_fiber_diameters(IMAGE_FOLDER, output_csv=RESULTS_CSV)
    print(f"Análisis completado! Resultados guardados en {RESULTS_CSV}")
    print(f"Resumen estadístico:\n{df_results.describe()}")

No está mal el resultado me falta información de saber si se está ajustando bien a nuestro caso

In [ ]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
from skimage import measure
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

def estimate_diameters_by_cross_section(binary_image, step=10, window=5):
    """
    Estima diámetros a lo largo del esqueleto usando líneas perpendiculares.
    """
    # Esqueletizar
    skeleton = skeletonize(binary_image)
    yx_coords = np.column_stack(np.where(skeleton))

    # Suavizar contorno para estimar dirección
    smoothed = gaussian_filter(skeleton.astype(float), sigma=2)
    gy, gx = np.gradient(smoothed)
    
    diameters = []
    for i in range(0, len(yx_coords), step):
        y, x = yx_coords[i]
        
        # Dirección perpendicular
        dx = -gy[y, x]
        dy = gx[y, x]
        norm = np.hypot(dx, dy)
        if norm == 0:
            continue
        dx /= norm
        dy /= norm
        
        # Trazar línea en ambas direcciones desde el punto del esqueleto
        length = 0
        for sign in [-1, 1]:
            for l in range(1, 50):  # máx 50 píxeles hacia afuera
                xi = int(round(x + sign * dx * l))
                yi = int(round(y + sign * dy * l))
                if 0 <= xi < binary_image.shape[1] and 0 <= yi < binary_image.shape[0]:
                    if not binary_image[yi, xi]:  # alcanzó el borde
                        break
                else:
                    break
                length += 1
        
        diameter = length
        if diameter > 0:
            diameters.append(diameter)

    return np.array(diameters)

# ==== USO ====
if __name__ == "__main__":
    img = cv2.imread("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au\\IVM4101#_TI_25XW_Rhodopsin_Au.png", cv2.IMREAD_GRAYSCALE)
    blurred = cv2.GaussianBlur(img, (5,5), 0)
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary = binary.astype(bool)

    diameters = estimate_diameters_by_cross_section(binary)
    
    plt.hist(diameters, bins=20)
    plt.title("Distribución de Diámetros Estimados")
    plt.xlabel("Diámetro (px)")
    plt.ylabel("Frecuencia")
    plt.show()


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
from scipy.ndimage import gaussian_filter

def visualize_fiber_diameters(image, binary, step=15, window=5, max_length=50, save_path=None):
    """
    Visualiza el esqueleto y los diámetros estimados sobre la imagen original.
    """
    skeleton = skeletonize(binary)
    yx_coords = np.column_stack(np.where(skeleton))
    smoothed = gaussian_filter(skeleton.astype(float), sigma=2)
    gy, gx = np.gradient(smoothed)

    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(image, cmap='gray')
    ax.imshow(skeleton, cmap='Reds', alpha=0.4)
    
    for i in range(0, len(yx_coords), step):
        y, x = yx_coords[i]
        dx = -gy[y, x]
        dy = gx[y, x]
        norm = np.hypot(dx, dy)
        if norm == 0:
            continue
        dx /= norm
        dy /= norm

        # Buscar borde en ambas direcciones
        length_pos = 0
        for l in range(1, max_length):
            xi = int(round(x + dx * l))
            yi = int(round(y + dy * l))
            if 0 <= xi < binary.shape[1] and 0 <= yi < binary.shape[0]:
                if not binary[yi, xi]:
                    break
            else:
                break
            length_pos += 1

        length_neg = 0
        for l in range(1, max_length):
            xi = int(round(x - dx * l))
            yi = int(round(y - dy * l))
            if 0 <= xi < binary.shape[1] and 0 <= yi < binary.shape[0]:
                if not binary[yi, xi]:
                    break
            else:
                break
            length_neg += 1

        # Coordenadas de los extremos de la línea de diámetro
        x1 = x - dx * length_neg
        y1 = y - dy * length_neg
        x2 = x + dx * length_pos
        y2 = y + dy * length_pos

        ax.plot([x1, x2], [y1, y2], color='lime', linewidth=2, alpha=0.7)
        ax.scatter([x], [y], color='blue', s=10)

    ax.set_title("Validación visual de diámetros de fibras")
    ax.axis('off')
    if save_path:
        plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.show()

# ==== USO EN UN STACK ====
folder = r"C:\\Users\\danwo\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au"
files = sorted([f for f in os.listdir(folder) if f.endswith(('.png','.tif','.jpg'))])

for idx, fname in enumerate(files[:5]):  # Cambia el rango para más imágenes
    img_path = os.path.join(folder, fname)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    blurred = cv2.GaussianBlur(img, (5,5), 0)
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary = binary.astype(bool)
    print(f"Mostrando validación para: {fname}")
    visualize_fiber_diameters(img, binary, step=20)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize, remove_small_objects
from scipy.ndimage import distance_transform_edt
from skimage import measure, color, segmentation
import pandas as pd
import tqdm
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter


def show_diameters_oriented(img, skeleton, diameters, step=20, max_len=300, save_path=None):
    """
    Dibuja sobre la imagen original las líneas de diámetro orientadas perpendicularmente al eje local.
    
    :param img: 2D array, imagen original en gris.
    :param skeleton: bool array, máscara de esqueleto.
    :param diameters: float array, 2*radio en cada píxel del esqueleto.
    :param step: int, muestreo sobre los puntos del esqueleto.
    :param max_len: int, longitud máxima (en px) de la línea de diámetro.
    :param save_path: str o None, ruta donde guardar la figura.
    """
    # 1) Suavizamos el esqueleto para poder derivar
    sk_f = gaussian_filter(skeleton.astype(float), sigma=1.5)
    gy, gx = np.gradient(sk_f)   # (dy, dx) ≈ dirección tangente
    
    # 2) Preparamos plot
    yx = np.column_stack(np.where(skeleton))
    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(img, cmap='gray')
    ax.axis('off')
    ax.set_title("Diámetros orientados sobre la fibra")
    
    # 3) Recorremos con un stride para no saturar
    for (y, x) in yx[::step]:
        d = diameters[y, x]
        if d <= 0: 
            continue
        
        # 4) La tangente viene de (gx, gy); la normal es (-gy, gx)
        tx, ty = gx[y, x], gy[y, x]
        norm = np.hypot(tx, ty)
        if norm == 0:
            continue
        tx, ty = tx/norm, ty/norm
        nx, ny = -ty, tx
        
        # 5) Definimos medio-largo de la línea
        half = min(d/2, max_len/2)

        # 6) Endpoints centrados en (x,y)
        x0, y0 = x - nx*half, y - ny*half
        x1, y1 = x + nx*half, y + ny*half
        
        # 7) Dibujamos en cyan
        ax.plot([x0, x1], [y0, y1], color='cyan', linewidth=1)
    
    if save_path:
        plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.show()


def analyze_fiber_diameters(image_folder, output_csv="fiber_diameters.csv", 
                           min_diameter=3, max_diameter=100,
                           viz_folder="fiber_viz"):
    """
    Analiza automáticamente diámetros de fibras con visualización
    """
    # Crear carpeta para visualizaciones
    os.makedirs(viz_folder, exist_ok=True)
    
    results = []
    valid_extensions = ('.tif', '.tiff', '.png', '.jpg', '.jpeg')
    
    # Procesar cada imagen
    for filename in tqdm.tqdm(os.listdir(image_folder)):
        if not filename.lower().endswith(valid_extensions):
            continue
            
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            continue
            
        # Preprocesamiento
        denoised = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)
        equalized = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)).apply(denoised)
        blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
        

        #Pipeline si no se invierte detecta las fibras como regiones oscuras debido (cv2.ADAPTIVE_THRESH_GAUSSIAN_C con cv2.THRESH_BINARY_INV)
        #Es por ello que es necesario en nuestro caso invertir la imagen antes de binarizarla.
        # INVERTIR la imagen para que las fibras brillantes sean primer plano
        inverted = cv2.bitwise_not(blurred)

        # Binarización adaptativa
        binary = cv2.adaptiveThreshold(
            inverted, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, 51, 7
        )
        




        # Limpieza morfológica
        kernel = np.ones((3,3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Eliminar objetos pequeños
        cleaned_bool = cleaned.astype(bool)
        cleaned_bool = remove_small_objects(cleaned_bool, min_size=50)
        cleaned = cleaned_bool.astype(np.uint8) * 255
        
        # Esqueletización y transformada de distancia
        skeleton = skeletonize(cleaned_bool)
        dist_transform = distance_transform_edt(cleaned_bool)
        skeleton_dist = dist_transform * skeleton
        
        # Calcular diámetros (2 * radio)
        diameters = 2 * skeleton_dist
        
        # Filtrar diámetros válidos
        valid_mask = (diameters > min_diameter) & (diameters < max_diameter) & skeleton
        valid_diameters = diameters[valid_mask]
        
        
        
        # Generar visualización de diámetros superpuestos
        viz_diam_path = os.path.join(viz_folder, f"viz_diametros_{os.path.splitext(filename)[0]}.png")
        show_diameters_oriented(img, skeleton, diameters, step=20, save_path=viz_diam_path)
        
        
        # Estadísticas
        if len(valid_diameters) > 0:
            stats = {
                'filename': filename,
                'mean_diameter': np.mean(valid_diameters),
                'median_diameter': np.median(valid_diameters),
                'std_diameter': np.std(valid_diameters),
                'min_diameter': np.min(valid_diameters),
                'max_diameter': np.max(valid_diameters),
                'fiber_count': len(valid_diameters),
                'viz_diam_path': viz_diam_path
            }
            results.append(stats)
        else:
            stats = {
                'filename': filename,
                'mean_diameter': np.nan,
                'median_diameter': np.nan,
                'std_diameter': np.nan,
                'min_diameter': np.nan,
                'max_diameter': np.nan,
                'fiber_count': 0,
                'viz_diam_path': viz_diam_path
            }
            results.append(stats)
    
    # Generar reporte
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    
    # Generar reporte global
    if not df.empty and df['fiber_count'].sum() > 0:
        plt.figure(figsize=(14, 10))
        
        # Histograma global de diámetros
        all_diameters = np.concatenate([df[df['fiber_count'] > 0]['mean_diameter'].values])
        plt.subplot(2, 2, 1)
        plt.hist(all_diameters, bins=50, alpha=0.7, color='purple')
        plt.title('Distribución Global de Diámetros Promedio')
        plt.xlabel('Diámetro (px)')
        plt.grid(True)
        
        # Distribución por imagen
        plt.subplot(2, 2, 2)
        plt.scatter(df.index, df['mean_diameter'], c=df['fiber_count'], cmap='viridis', s=100)
        plt.colorbar(label='Número de Fibras')
        plt.title('Diámetro Promedio por Imagen')
        plt.ylabel('Diámetro (px)')
        plt.grid(True)
        
        # Boxplot
        plt.subplot(2, 2, 3)
        valid_df = df[df['fiber_count'] > 0]
        plt.boxplot([valid_df['mean_diameter'], valid_df['median_diameter']], 
                   labels=['Media', 'Mediana'])
        plt.title('Distribución de Medidas')
        plt.ylabel('Diámetro (px)')
        plt.grid(True)
        
        # Relación conteo-diámetro
        plt.subplot(2, 2, 4)
        plt.scatter(valid_df['mean_diameter'], valid_df['fiber_count'], alpha=0.6)
        plt.title('Relación Diámetro vs Cantidad de Fibras')
        plt.xlabel('Diámetro Promedio (px)')
        plt.ylabel('Cantidad de Fibras')
        plt.grid(True)
        
        plt.tight_layout()
        plt.savefig('global_fiber_analysis.png', dpi=200)
    
    return df

# Uso =============================================
if __name__ == "__main__":
    # Configuración
    IMAGE_FOLDER = r"C:\\Users\\danwo\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au"
    RESULTS_CSV = "resultados_diametros.csv"
    VIZ_FOLDER = "visualizaciones"
    
    # Ejecutar análisis
    df_results = analyze_fiber_diameters(
        IMAGE_FOLDER, 
        output_csv=RESULTS_CSV,
        min_diameter=3,
        max_diameter=100,
        viz_folder=VIZ_FOLDER
    )
    
    print(f"Análisis completado!")
    print(f"- Imágenes procesadas: {len(df_results)}")
    print(f"- Visualizaciones guardadas en: {VIZ_FOLDER}")
    print(f"- Resultados cuantitativos en: {RESULTS_CSV}")

AQUI YA ESTÁ BIEN REPRESENTADO Y PODEMOS VER QUE LOS DIÁMETROS EN LÍNEAS GENERALES SE AJUSTAN BIEN A LAS FIBRAS. LO ÚNICO QUE AHORA HAY QUE CAMBIAR TODO LO QUE HE HECHO ANTES E INVERTIRLO PORQUE LO QUE SE ESTÁ ANALIZANDO SINO SON FIBRAS OSCURAS EN VEZ DE CLARAS

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize, remove_small_objects
from scipy.ndimage import distance_transform_edt, gaussian_filter
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
import tqdm

def show_diameters_oriented(img, skeleton, diameters, step=20, max_len=50, save_path=None):
    """
    Dibuja sobre la imagen original las líneas de diámetro orientadas 
    perpendicularmente al eje local, y el esqueleto en semitransparente.
    """
    # suavizar el esqueleto para obtener gradientes
    sk_f = gaussian_filter(skeleton.astype(float), sigma=1.5)
    gy, gx = np.gradient(sk_f)   # gradientes tangentes
    
    ys, xs = np.where(skeleton)
    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(img, cmap='gray')
    # esqueleto semitransparente en rojo
    ax.scatter(xs, ys, s=1, c='red', alpha=0.3, label='esqueleto')
    
    for (y, x) in zip(ys[::step], xs[::step]):
        d = diameters[y, x]
        if d <= 0: 
            continue
        
        # tangente (gx, gy) → normal perpendicular (-gy, gx)
        tx, ty = gx[y, x], gy[y, x]
        n = np.hypot(tx, ty)
        if n == 0:
            continue
        tx, ty = tx/n, ty/n
        nx, ny = -ty, tx
        
        # longitud media de la línea
        half = min(d/2, max_len/2)
        x0, y0 = x - nx*half, y - ny*half
        x1, y1 = x + nx*half, y + ny*half
        
        ax.plot([x0, x1], [y0, y1], color='cyan', linewidth=1, alpha=0.8)
    
    ax.set_title("Diámetros orientados sobre la fibra")
    ax.axis('off')
    ax.legend(loc='lower right')
    if save_path:
        plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.show()
    plt.close(fig)

def visualize_fiber_analysis(img, cleaned, skeleton, diameters, output_path):
    """
    Genera un panel de 6 subplots para diagnóstico detallado.
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Análisis de Diámetros de Fibras', fontsize=18)
    
    # Colormap suave
    cmap = LinearSegmentedColormap.from_list('diam_cmap',
                                             ['navy','blue','cyan','yellow','orange','red'])
    
    # normalizar
    vals = diameters[diameters>0]
    vmin, vmax = (np.percentile(vals,5), np.percentile(vals,95)) if vals.size else (0,1)
    
    # 1) original + esqueleto
    axes[0,0].imshow(img, cmap='gray')
    axes[0,0].contour(skeleton, colors='r', linewidths=0.5, alpha=0.6)
    axes[0,0].set_title('Original + Esqueleto')
    axes[0,0].axis('off')
    
    # 2) segmentación limpia
    axes[0,1].imshow(cleaned, cmap='gray')
    axes[0,1].set_title('Segmentación y Limpieza')
    axes[0,1].axis('off')
    
    # 3) mapa de calor sobre RGB
    rgb = np.zeros((*img.shape,3),dtype=float)
    rgb[...,0] = img/255.           # fondo gris en rojo
    rgb[skeleton,1] = 1.            # esqueleto en verde
    norm = np.clip((diameters-vmin)/(vmax-vmin),0,1)
    heat = cmap(norm)[...,:3]
    rgb[...,2] = heat[...,0]        # componentes azules
    axes[0,2].imshow(rgb)
    axes[0,2].set_title('Esqueleto + Mapa de Diámetros')
    axes[0,2].axis('off')
    
    # 4) histograma de diámetros
    if vals.size:
        axes[1,0].hist(vals, bins=40, alpha=0.8, color='teal')
        axes[1,0].axvline(vals.mean(), color='magenta', linestyle='--', label=f"μ={vals.mean():.1f}")
        axes[1,0].set_title('Distribución de Diámetros')
        axes[1,0].set_xlabel('Diámetro (px)')
        axes[1,0].set_ylabel('Frecuencia')
        axes[1,0].legend()
        axes[1,0].grid(True)
    
    # 5) fibras coloreadas según diámetro
    fiber_map = np.zeros_like(img, dtype=float)
    fiber_map[skeleton] = diameters[skeleton]
    fiber_map = cv2.GaussianBlur(fiber_map, (5,5), 0)
    im5 = axes[1,1].imshow(img, cmap='gray')
    im5 = axes[1,1].imshow(fiber_map, cmap=cmap, alpha=0.6, vmin=vmin, vmax=vmax)
    axes[1,1].set_title('Fibras coloreadas por diámetro')
    axes[1,1].axis('off')
    plt.colorbar(im5, ax=axes[1,1], fraction=0.046, pad=0.04, label='px')
    
    # 6) zoom con etiquetas
    ys, xs = np.where(skeleton)
    if xs.size:
        cx, cy = np.median(xs), np.median(ys)
        z = int(min(img.shape)*0.2)
        y0,y1 = int(cy-z/2), int(cy+z/2)
        x0,x1 = int(cx-z/2), int(cx+z/2)
        zi = np.clip(img[y0:y1, x0:x1],0,255)
        zs = skeleton[y0:y1, x0:x1]
        axes[1,2].imshow(zi, cmap='gray')
        axes[1,2].contour(zs, colors='yellow', linewidths=0.6)
        ys2, xs2 = np.where(zs)
        for (y,x) in zip(ys2, xs2):
            d = diameters[y0+y, x0+x]
            if d>0 and np.random.rand()<0.2:
                axes[1,2].text(x, y, f"{d:.1f}", color='cyan',
                               fontsize=6, ha='center', va='center')
        axes[1,2].set_title('Zoom con diámetros')
        axes[1,2].axis('off')
    
    plt.tight_layout(rect=[0,0,1,0.96])
    plt.savefig(output_path, dpi=150)
    plt.close(fig)

def analyze_fiber_diameters(image_folder, output_csv="fiber_diameters.csv",
                            min_diameter=3, max_diameter=100,
                            viz_folder="fiber_viz"):
    os.makedirs(viz_folder, exist_ok=True)
    results = []
    exts = ('.tif','.tiff','.png','.jpg','.jpeg')
    
    for fname in tqdm.tqdm(sorted(os.listdir(image_folder))):
        if not fname.lower().endswith(exts):
            continue
        path = os.path.join(image_folder, fname)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        
        # --- Preprocesado ---
        den = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)).apply(den)
        blur = cv2.GaussianBlur(clahe, (5,5), 0)
        binary = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV,51,7)
        m = np.ones((3,3),np.uint8)
        clean = cv2.morphologyEx(binary, cv2.MORPH_OPEN, m, iterations=1)
        clean = cv2.morphologyEx(clean, cv2.MORPH_CLOSE, m, iterations=2)
        
        clean_bool = remove_small_objects(clean.astype(bool), min_size=50)
        clean = clean_bool.astype(np.uint8)*255
        
        # --- Esqueleto y diámetros ---
        skel = skeletonize(clean_bool)
        dist = distance_transform_edt(clean_bool)
        diams = 2 * dist * skel
        
        mask = (diams>min_diameter)&(diams<max_diameter)&skel
        vals = diams[mask]
        
        # rutas de guardado
        base = os.path.splitext(fname)[0]
        p1 = os.path.join(viz_folder, f"panel_{base}.png")
        p2 = os.path.join(viz_folder, f"lines_{base}.png")
        
        visualize_fiber_analysis(img, clean, skel, diams, p1)
        show_diameters_oriented(img, skel, diams, step=25, max_len=60, save_path=p2)
        
        stats = {
            'filename': fname,
            'mean_diameter': float(np.mean(vals)) if vals.size else np.nan,
            'median_diameter': float(np.median(vals)) if vals.size else np.nan,
            'std_diameter': float(np.std(vals)) if vals.size else np.nan,
            'min_diameter': float(np.min(vals)) if vals.size else np.nan,
            'max_diameter': float(np.max(vals)) if vals.size else np.nan,
            'fiber_count': int(vals.size),
            'panel_viz': p1,
            'lines_viz': p2
        }
        results.append(stats)
    
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    
    # reporte global (opcional, similar al anterior)
    return df

if __name__ == "__main__":
    IMAGE_FOLDER = r"C:\Users\danwo\Desktop\ZS0001_TI_25XW_Rhodopsin_Au"
    RESULTS_CSV   = "resultados_diametros.csv"
    VIZ_FOLDER    = "visualizaciones"
    
    df = analyze_fiber_diameters(IMAGE_FOLDER,
                                 output_csv=RESULTS_CSV,
                                 min_diameter=3,
                                 max_diameter=100,
                                 viz_folder=VIZ_FOLDER)
    print("Análisis completado:")
    print(df.describe().T)  # breve resumen    


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize, remove_small_objects, remove_small_holes
from skimage.segmentation import watershed
from scipy.ndimage import distance_transform_edt, gaussian_filter
from skimage import measure, color, segmentation, filters
from skimage.feature import peak_local_max
import pandas as pd
import tqdm
from matplotlib.colors import LinearSegmentedColormap

# Funciones de visualización
def show_diameters_oriented(img, skeleton, diameters, step=20, max_len=50, save_path=None):
    """
    Dibuja sobre la imagen original las líneas de diámetro orientadas perpendicularmente al eje local.
    
    :param img: 2D array, imagen original en gris.
    :param skeleton: bool array, máscara de esqueleto.
    :param diameters: float array, 2*radio en cada píxel del esqueleto.
    :param step: int, muestreo sobre los puntos del esqueleto.
    :param max_len: int, longitud máxima (en px) de la línea de diámetro.
    :param save_path: str o None, ruta donde guardar la figura.
    """
    # 1) Suavizamos el esqueleto para poder derivar
    sk_f = gaussian_filter(skeleton.astype(float), sigma=1.5)
    gy, gx = np.gradient(sk_f)   # (dy, dx) ≈ dirección tangente
    
    # 2) Preparamos plot
    yx = np.column_stack(np.where(skeleton))
    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(img, cmap='gray')
    ax.axis('off')
    ax.set_title("Diámetros orientados sobre la fibra")
    
    # 3) Recorremos con un stride para no saturar
    for (y, x) in yx[::step]:
        d = diameters[y, x]
        if d <= 0: 
            continue
        
        # 4) La tangente viene de (gx, gy); la normal es (-gy, gx)
        tx, ty = gx[y, x], gy[y, x]
        norm = np.hypot(tx, ty)
        if norm == 0:
            continue
        tx, ty = tx/norm, ty/norm
        nx, ny = -ty, tx
        
        # 5) Definimos medio-largo de la línea
        half = min(d/2, max_len/2)
        
        # 6) Endpoints centrados en (x,y)
        x0, y0 = x - nx*half, y - ny*half
        x1, y1 = x + nx*half, y + ny*half
        
        # 7) Dibujamos en cyan
        ax.plot([x0, x1], [y0, y1], color='cyan', linewidth=1)
    
    if save_path:
        plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.show()

def visualize_fiber_analysis(img, cleaned, skeleton, diameters, output_path):
    """
    Crea una visualización completa del análisis de fibras
    """
    # Crear una imagen compuesta para diagnóstico
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Análisis de Diámetros de Fibras', fontsize=16)
    
    # 1. Imagen original
    axes[0, 0].imshow(img, cmap='gray')
    axes[0, 0].set_title('Imagen Original')
    
    # 2. Imagen segmentada y limpiada
    axes[0, 1].imshow(cleaned, cmap='gray')
    axes[0, 1].set_title('Segmentación y Limpieza')
    
    # 3. Esqueletos con mapa de calor de diámetros
    # Crear un mapa de colores personalizado (azul para valores bajos, rojo para altos)
    cmap = LinearSegmentedColormap.from_list('diameter_cmap', ['blue', 'green', 'yellow', 'red'])
    
    # Normalizar diámetros para el mapa de colores
    valid_diameters = diameters[diameters > 0]
    if len(valid_diameters) > 0:
        vmin, vmax = np.percentile(valid_diameters, [5, 95])
    else:
        vmin, vmax = 0, 1
    
    # Crear imagen RGB para visualización
    diag_img = np.zeros((*skeleton.shape, 3))
    diag_img[..., 0] = img / 255.0  # Canal rojo: imagen original
    
    # Canal verde: esqueletos
    diag_img[skeleton, 1] = 1.0
    
    # Canal azul: mapa de calor de diámetros
    norm_diameters = (diameters - vmin) / (vmax - vmin)
    norm_diameters = np.clip(norm_diameters, 0, 1)
    diag_img[..., 2] = cmap(norm_diameters)[..., 0]  # Usar componente roja del colormap
    
    axes[0, 2].imshow(diag_img)
    axes[0, 2].set_title('Esqueletos y Diámetros (Azul: Original, Verde: Esqueleto, Rojo: Diámetro)')
    
    # 4. Distribución de diámetros
    if len(valid_diameters) > 0:
        axes[1, 0].hist(valid_diameters, bins=50, alpha=0.7, color='blue')
        axes[1, 0].axvline(np.mean(valid_diameters), color='red', linestyle='dashed', linewidth=1)
        axes[1, 0].set_xlabel('Diámetro (píxeles)')
        axes[1, 0].set_ylabel('Frecuencia')
        axes[1, 0].set_title(f'Distribución de Diámetros\nMedia: {np.mean(valid_diameters):.2f} px')
        axes[1, 0].grid(True)
    
    # 5. Imagen con fibras coloreadas por diámetro
    if len(valid_diameters) > 0:
        # Crear máscara de fibras con valores de diámetro
        fiber_map = np.zeros_like(img, dtype=float)
        fiber_map[skeleton] = diameters[skeleton]
        
        # Suavizar para mejor visualización
        fiber_map = cv2.GaussianBlur(fiber_map, (5, 5), 0)
        
        axes[1, 1].imshow(img, cmap='gray')
        im = axes[1, 1].imshow(fiber_map, alpha=0.6, cmap=cmap, vmin=vmin, vmax=vmax)
        axes[1, 1].set_title('Fibras Coloreadas por Diámetro')
        
        # Añadir barra de colores
        plt.colorbar(im, ax=axes[1, 1], label='Diámetro (píxeles)')
    
    # 6. Zoom en una región interesante (autodetecta área con alta densidad)
    if np.any(skeleton):
        # Encontrar región con alta densidad de esqueletos
        y, x = np.where(skeleton)
        if len(x) > 0 and len(y) > 0:
            center_x, center_y = np.median(x), np.median(y)
            
            # Tamaño del zoom (20% de la imagen)
            zoom_size = int(min(img.shape) * 0.2)
            
            # Coordenadas del recorte
            y_start = int(max(0, center_y - zoom_size/2))
            y_end = int(min(img.shape[0], center_y + zoom_size/2))
            x_start = int(max(0, center_x - zoom_size/2))
            x_end = int(min(img.shape[1], center_x + zoom_size/2))
            
            # Aplicar zoom
            zoom_img = img[y_start:y_end, x_start:x_end]
            zoom_skel = skeleton[y_start:y_end, x_start:x_end]
            
            # Visualizar con diámetros superpuestos
            axes[1, 2].imshow(zoom_img, cmap='gray')
            axes[1, 2].contour(zoom_skel, colors='yellow', linewidths=0.5)
            
            # Añadir etiquetas de diámetro
            ys, xs = np.where(zoom_skel)
            for yi, xi in zip(ys, xs):
                diam = diameters[y_start+yi, x_start+xi]
                if diam > 0:
                    # Mostrar solo cada 5to píxel para no saturar
                    if np.random.random() < 0.2:
                        axes[1, 2].text(xi, yi, f'{diam:.1f}', 
                                       color='cyan', fontsize=6, 
                                       ha='center', va='center')
            
            axes[1, 2].set_title('Zoom con Diámetros (px)')
    
    # Ajustes finales
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(output_path, dpi=150)
    plt.close(fig)

def analyze_fiber_diameters(image_folder, output_csv="fiber_diameters.csv", 
                           min_diameter=3, max_diameter=100,
                           viz_folder="fiber_viz"):
    """
    Analiza automáticamente diámetros de fibras con visualización
    """
    # Crear carpeta para visualizaciones
    os.makedirs(viz_folder, exist_ok=True)
    
    results = []
    valid_extensions = ('.tif', '.tiff', '.png', '.jpg', '.jpeg')
    
    # Procesar cada imagen
    for filename in tqdm.tqdm(os.listdir(image_folder)):
        if not filename.lower().endswith(valid_extensions):
            continue
            
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            continue
            
        # Preprocesamiento
        denoised = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
        unsharp_mask = cv2.addWeighted(img, 1.5, denoised, -0.5, 0)
        equalized = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)).apply(unsharp_mask)
        blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
        
        # Binarización adaptativa
        binary = filters.threshold_local(blurred, block_size=51, offset=7)
        binary = (blurred > binary).astype(np.uint8) * 255
        
        # Limpieza morfológica
        kernel = np.ones((3,3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Eliminar objetos pequeños y agujeros
        cleaned_bool = cleaned.astype(bool)
        cleaned_bool = remove_small_objects(cleaned_bool, min_size=50)
        cleaned_bool = remove_small_holes(cleaned_bool, area_threshold=50)
        cleaned = cleaned_bool.astype(np.uint8) * 255
        
        # Separación de fibras superpuestas con watershed
        distance = distance_transform_edt(cleaned_bool)
        # Cambiar esta línea:
        local_maxi = peak_local_max(distance, labels=cleaned_bool, footprint=np.ones((3, 3)), exclude_border=False)
        if isinstance(local_maxi, np.ndarray) and local_maxi.dtype == bool:
            # Ya es un mapa booleano (scikit-image >=0.20)
            pass
        else:
            # Es una lista de coordenadas (scikit-image <0.20), hay que convertirlo a máscara
            mask = np.zeros_like(distance, dtype=bool)
            for y, x in local_maxi:
                mask[y, x] = True
            local_maxi = mask       
        markers = measure.label(local_maxi)
        labels = watershed(-distance, markers, mask=cleaned_bool)
        
        # Esqueletización y transformada de distancia
        skeleton = skeletonize(cleaned_bool)
        dist_transform = distance_transform_edt(cleaned_bool)
        skeleton_dist = dist_transform * skeleton
        
        # Calcular diámetros (2 * radio)
        diameters = 2 * skeleton_dist
        
        # Filtrar diámetros válidos
        valid_mask = (diameters > min_diameter) & (diameters < max_diameter) & skeleton
        valid_diameters = diameters[valid_mask]
        
         # Generar visualización principal
        viz_path = os.path.join(viz_folder, f"viz_{os.path.splitext(filename)[0]}.png")
        visualize_fiber_analysis(img, cleaned, skeleton, diameters, viz_path)
        
        # Generar visualización de diámetros superpuestos
        viz_diam_path = os.path.join(viz_folder, f"viz_diametros_{os.path.splitext(filename)[0]}.png")
        show_diameters_oriented(img, skeleton, diameters, step=20, save_path=viz_diam_path)
        
        
        # Estadísticas
        if len(valid_diameters) > 0:
            stats = {
                'filename': filename,
                'mean_diameter': np.mean(valid_diameters),
                'median_diameter': np.median(valid_diameters),
                'std_diameter': np.std(valid_diameters),
                'min_diameter': np.min(valid_diameters),
                'max_diameter': np.max(valid_diameters),
                'fiber_count': len(valid_diameters),
                'viz_path': viz_path,
                'viz_diam_path': viz_diam_path
            }
            results.append(stats)
        else:
            stats = {
                'filename': filename,
                'mean_diameter': np.nan,
                'median_diameter': np.nan,
                'std_diameter': np.nan,
                'min_diameter': np.nan,
                'max_diameter': np.nan,
                'fiber_count': 0,
                'viz_path': viz_path,
                'viz_diam_path': viz_diam_path
            }
            results.append(stats)
    
    # Generar reporte
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    
    # Generar reporte global
    if not df.empty and df['fiber_count'].sum() > 0:
        plt.figure(figsize=(14, 10))
        
        # Histograma global de diámetros
        all_diameters = np.concatenate([df[df['fiber_count'] > 0]['mean_diameter'].values])
        plt.subplot(2, 2, 1)
        plt.hist(all_diameters, bins=50, alpha=0.7, color='purple')
        plt.title('Distribución Global de Diámetros Promedio')
        plt.xlabel('Diámetro (px)')
        plt.grid(True)
        
        # Distribución por imagen
        plt.subplot(2, 2, 2)
        plt.scatter(df.index, df['mean_diameter'], c=df['fiber_count'], cmap='viridis', s=100)
        plt.colorbar(label='Número de Fibras')
        plt.title('Diámetro Promedio por Imagen')
        plt.ylabel('Diámetro (px)')
        plt.grid(True)
        
        # Boxplot
        plt.subplot(2, 2, 3)
        valid_df = df[df['fiber_count'] > 0]
        plt.boxplot([valid_df['mean_diameter'], valid_df['median_diameter']], 
                   labels=['Media', 'Mediana'])
        plt.title('Distribución de Medidas')
        plt.ylabel('Diámetro (px)')
        plt.grid(True)
        
        # Relación conteo-diámetro
        plt.subplot(2, 2, 4)
        plt.scatter(valid_df['mean_diameter'], valid_df['fiber_count'], alpha=0.6)
        plt.title('Relación Diámetro vs Cantidad de Fibras')
        plt.xlabel('Diámetro Promedio (px)')
        plt.ylabel('Cantidad de Fibras')
        plt.grid(True)
        
        plt.tight_layout()
        plt.savefig('global_fiber_analysis.png', dpi=200)
    
    return df

# Uso =============================================
if __name__ == "__main__":
    # Configuración
    IMAGE_FOLDER = r"C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au"
    RESULTS_CSV = "resultados_diametros.csv"
    VIZ_FOLDER = "visualizaciones"
    
    # Ejecutar análisis
    df_results = analyze_fiber_diameters(
        IMAGE_FOLDER, 
        output_csv=RESULTS_CSV,
        min_diameter=3,
        max_diameter=100,
        viz_folder=VIZ_FOLDER
    )
    
    print(f"Análisis completado!")
    print(f"- Imágenes procesadas: {len(df_results)}")
    print(f"- Visualizaciones guardadas en: {VIZ_FOLDER}")
    print(f"- Resultados cuantitativos en: {RESULTS_CSV}")


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize, remove_small_objects, remove_small_holes
from scipy.ndimage import distance_transform_edt, gaussian_filter
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
import tqdm

def show_diameters_oriented(img, skeleton, diameters, step=20, max_len=50, save_path=None):
    sk_f = gaussian_filter(skeleton.astype(float), sigma=1.5)
    gy, gx = np.gradient(sk_f)
    yx = np.column_stack(np.where(skeleton))
    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(img, cmap='gray')
    ax.axis('off')
    ax.set_title("Diámetros perpendiculares a la fibra")
    for (y, x) in yx[::step]:
        d = diameters[y, x]
        if d <= 0: continue
        tx, ty = gx[y, x], gy[y, x]
        norm = np.hypot(tx, ty)
        if norm == 0: continue
        tx, ty = tx/norm, ty/norm
        nx, ny = -ty, tx
        half = min(d/2, max_len/2)
        x0, y0 = x - nx*half, y - ny*half
        x1, y1 = x + nx*half, y + ny*half
        ax.plot([x0, x1], [y0, y1], color='cyan', linewidth=1)
    if save_path:
        plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.show()
    plt.close(fig)

def visualize_fiber_analysis(img, cleaned, skeleton, diameters, output_path):
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Análisis de Diámetros de Fibras', fontsize=16)
    cmap = LinearSegmentedColormap.from_list('diameter_cmap', ['blue', 'green', 'yellow', 'red'])
    valid_diameters = diameters[diameters > 0]
    vmin, vmax = (np.percentile(valid_diameters, [5, 95]) if len(valid_diameters) > 0 else (0, 1))
    diag_img = np.zeros((*skeleton.shape, 3))
    diag_img[..., 0] = img / 255.0
    diag_img[skeleton, 1] = 1.0
    norm_diameters = (diameters - vmin) / (vmax - vmin)
    norm_diameters = np.clip(norm_diameters, 0, 1)
    diag_img[..., 2] = cmap(norm_diameters)[..., 0]
    axes[0, 0].imshow(img, cmap='gray')
    axes[0, 0].set_title('Imagen Original')
    axes[0, 1].imshow(cleaned, cmap='gray')
    axes[0, 1].set_title('Segmentación y Limpieza')
    axes[0, 2].imshow(diag_img)
    axes[0, 2].set_title('Esqueleto y Diámetros')
    if len(valid_diameters) > 0:
        axes[1, 0].hist(valid_diameters, bins=50, alpha=0.7, color='blue')
        axes[1, 0].axvline(np.mean(valid_diameters), color='red', linestyle='dashed', linewidth=1)
        axes[1, 0].set_xlabel('Diámetro (px)')
        axes[1, 0].set_ylabel('Frecuencia')
        axes[1, 0].set_title(f'Distribución de Diámetros\nMedia: {np.mean(valid_diameters):.2f} px')
        axes[1, 0].grid(True)
        fiber_map = np.zeros_like(img, dtype=float)
        fiber_map[skeleton] = diameters[skeleton]
        fiber_map = cv2.GaussianBlur(fiber_map, (5, 5), 0)
        axes[1, 1].imshow(img, cmap='gray')
        im = axes[1, 1].imshow(fiber_map, alpha=0.6, cmap=cmap, vmin=vmin, vmax=vmax)
        axes[1, 1].set_title('Fibras por diámetro')
        plt.colorbar(im, ax=axes[1, 1], label='Diámetro (px)')
    if np.any(skeleton):
        y, x = np.where(skeleton)
        if len(x) > 0 and len(y) > 0:
            center_x, center_y = np.median(x), np.median(y)
            zoom_size = int(min(img.shape) * 0.2)
            y_start = int(max(0, center_y - zoom_size/2))
            y_end = int(min(img.shape[0], center_y + zoom_size/2))
            x_start = int(max(0, center_x - zoom_size/2))
            x_end = int(min(img.shape[1], center_x + zoom_size/2))
            zoom_img = img[y_start:y_end, x_start:x_end]
            zoom_skel = skeleton[y_start:y_end, x_start:x_end]
            axes[1, 2].imshow(zoom_img, cmap='gray')
            axes[1, 2].contour(zoom_skel, colors='yellow', linewidths=0.5)
            ys, xs = np.where(zoom_skel)
            for yi, xi in zip(ys, xs):
                diam = diameters[y_start+yi, x_start+xi]
                if diam > 0 and np.random.random() < 0.2:
                    axes[1, 2].text(xi, yi, f'{diam:.1f}', color='cyan', fontsize=6, ha='center', va='center')
            axes[1, 2].set_title('Zoom con Diámetros (px)')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(output_path, dpi=150)
    plt.close(fig)

def analyze_fiber_diameters(image_folder, output_csv="fiber_diameters.csv", min_diameter=3, max_diameter=100, viz_folder="fiber_viz"):
    os.makedirs(viz_folder, exist_ok=True)
    results = []
    valid_extensions = ('.tif', '.tiff', '.png', '.jpg', '.jpeg')
    for filename in tqdm.tqdm(os.listdir(image_folder)):
        if not filename.lower().endswith(valid_extensions):
            continue
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        # Preprocesamiento robusto
        denoised = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)).apply(denoised)
        blurred = cv2.GaussianBlur(clahe, (5, 5), 0)
        # Binarización adaptativa + Otsu
        binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51, 7)
        _, binary_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        binary = np.logical_or(binary, binary_otsu).astype(np.uint8) * 255
        # Limpieza morfológica
        kernel = np.ones((3,3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)
        cleaned_bool = cleaned.astype(bool)
        cleaned_bool = remove_small_objects(cleaned_bool, min_size=50)
        cleaned_bool = remove_small_holes(cleaned_bool, area_threshold=50)
        cleaned = cleaned_bool.astype(np.uint8) * 255
        # Esqueletización y transformada de distancia
        skeleton = skeletonize(cleaned_bool)
        dist_transform = distance_transform_edt(cleaned_bool)
        skeleton_dist = dist_transform * skeleton
        diameters = 2 * skeleton_dist
        valid_mask = (diameters > min_diameter) & (diameters < max_diameter) & skeleton
        valid_diameters = diameters[valid_mask]
        # Visualizaciones
        viz_path = os.path.join(viz_folder, f"viz_{os.path.splitext(filename)[0]}.png")
        visualize_fiber_analysis(img, cleaned, skeleton, diameters, viz_path)
        viz_diam_path = os.path.join(viz_folder, f"viz_diametros_{os.path.splitext(filename)[0]}.png")
        show_diameters_oriented(img, skeleton, diameters, step=20, save_path=viz_diam_path)
        # Estadísticas
        stats = {
            'filename': filename,
            'mean_diameter': np.mean(valid_diameters) if len(valid_diameters) else np.nan,
            'median_diameter': np.median(valid_diameters) if len(valid_diameters) else np.nan,
            'std_diameter': np.std(valid_diameters) if len(valid_diameters) else np.nan,
            'min_diameter': np.min(valid_diameters) if len(valid_diameters) else np.nan,
            'max_diameter': np.max(valid_diameters) if len(valid_diameters) else np.nan,
            'fiber_count': len(valid_diameters),
            'viz_path': viz_path,
            'viz_diam_path': viz_diam_path
        }
        results.append(stats)
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    # Resumen global
    if not df.empty and df['fiber_count'].sum() > 0:
        plt.figure(figsize=(14, 10))
        valid_df = df[df['fiber_count'] > 0]
        plt.subplot(2, 2, 1)
        plt.hist(valid_df['mean_diameter'], bins=50, alpha=0.7, color='purple')
        plt.title('Distribución Global de Diámetros Promedio')
        plt.xlabel('Diámetro (px)')
        plt.grid(True)
        plt.subplot(2, 2, 2)
        plt.scatter(valid_df.index, valid_df['mean_diameter'], c=valid_df['fiber_count'], cmap='viridis', s=100)
        plt.colorbar(label='Número de Fibras')
        plt.title('Diámetro Promedio por Imagen')
        plt.ylabel('Diámetro (px)')
        plt.grid(True)
        plt.subplot(2, 2, 3)
        plt.boxplot([valid_df['mean_diameter'], valid_df['median_diameter']], labels=['Media', 'Mediana'])
        plt.title('Distribución de Medidas')
        plt.ylabel('Diámetro (px)')
        plt.grid(True)
        plt.subplot(2, 2, 4)
        plt.scatter(valid_df['mean_diameter'], valid_df['fiber_count'], alpha=0.6)
        plt.title('Relación Diámetro vs Cantidad de Fibras')
        plt.xlabel('Diámetro Promedio (px)')
        plt.ylabel('Cantidad de Fibras')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig('global_fiber_analysis.png', dpi=200)

    return df

if __name__ == "__main__":
    IMAGE_FOLDER = r"C:\Users\danwo\Desktop\ZS0001_TI_25XW_Rhodopsin_Au"
    RESULTS_CSV = "resultados_diametros.csv"
    VIZ_FOLDER = "visualizaciones"
    df_results = analyze_fiber_diameters(
        IMAGE_FOLDER,
        output_csv=RESULTS_CSV,
        min_diameter=3,
        max_diameter=100,
        viz_folder=VIZ_FOLDER
    )
    print(f"Análisis completado!")
    print(f"- Imágenes procesadas: {len(df_results)}")
    print(f"- Visualizaciones guardadas en: {VIZ_FOLDER}")
    print(f"- Resultados cuantitativos en: {RESULTS_CSV}")

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize, remove_small_objects
from scipy.ndimage import distance_transform_edt
import pandas as pd
import tqdm
from scipy.stats import mode
from scipy.ndimage import gaussian_filter

def show_diameters_oriented(img, skeleton, diameters, step=20, max_len=300, save_path=None):
    sk_f = gaussian_filter(skeleton.astype(float), sigma=1.5)
    gy, gx = np.gradient(sk_f)
    yx = np.column_stack(np.where(skeleton))
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img, cmap='gray')
    ax.axis('off')
    ax.set_title("Diámetros orientados sobre la fibra")
    for (y, x) in yx[::step]:
        d = diameters[y, x]
        if d <= 0: continue
        tx, ty = gx[y, x], gy[y, x]
        norm = np.hypot(tx, ty)
        if norm == 0: continue
        tx, ty = tx/norm, ty/norm
        nx, ny = -ty, tx
        half = min(d/2, max_len/2)
        x0, y0 = x - nx*half, y - ny*half
        x1, y1 = x + nx*half, y + ny*half
        ax.plot([x0, x1], [y0, y1], color='cyan', linewidth=1)
    if save_path:
        plt.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.close()

def analyze_fiber_diameters(image_folder, output_csv="fiber_diameters_summary.csv", 
                            min_diameter=3, max_diameter=100, bin_size=5,
                            viz_folder="fiber_viz"):
    os.makedirs(viz_folder, exist_ok=True)
    summary_list = []
    detailed_list = []
    valid_extensions = ('.tif', '.tiff', '.png', '.jpg', '.jpeg')
    for filename in tqdm.tqdm(sorted(os.listdir(image_folder))):
        if not filename.lower().endswith(valid_extensions):
            continue
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None: continue
        denoised = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)
        equalized = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)).apply(denoised)
        blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
        inverted = cv2.bitwise_not(blurred)
        binary = cv2.adaptiveThreshold(inverted, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 51, 7)
        kernel = np.ones((3,3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)
        cleaned_bool = cleaned.astype(bool)
        cleaned_bool = remove_small_objects(cleaned_bool, min_size=50)
        cleaned = cleaned_bool.astype(np.uint8) * 255
        skeleton = skeletonize(cleaned_bool)
        dist_transform = distance_transform_edt(cleaned_bool)
        skeleton_dist = dist_transform * skeleton
        diameters = 2 * skeleton_dist
        valid_mask = (diameters > min_diameter) & (diameters < max_diameter) & skeleton
        valid_diameters = diameters[valid_mask]
        if valid_diameters.size == 0:
            continue
        mean_d = np.mean(valid_diameters)
        median_d = np.median(valid_diameters)
        std_d = np.std(valid_diameters)
        mode_res = mode(valid_diameters, keepdims=False)
        mode_d = mode_res.mode if valid_diameters.size > 0 else np.nan
        bin_edges = np.arange(min_diameter, max_diameter + bin_size, bin_size)
        bin_counts, _ = np.histogram(valid_diameters, bins=bin_edges)
        bin_labels = [f"{int(b1)}-{int(b2)}" for b1, b2 in zip(bin_edges[:-1], bin_edges[1:])]
        bin_data = dict(zip(bin_labels, bin_counts))
        viz_diam_path = os.path.join(viz_folder, f"viz_diametros_{os.path.splitext(filename)[0]}.png")
        show_diameters_oriented(img, skeleton, diameters, step=20, save_path=viz_diam_path)
        summary_row = {
            "filename": filename,
            "mean_diameter": mean_d,
            "median_diameter": median_d,
            "mode_diameter": float(mode_d),
            "std_diameter": std_d,
            "fiber_count": len(valid_diameters)
        }
        summary_row.update(bin_data)
        summary_list.append(summary_row)
        for d in valid_diameters:
            detailed_list.append({"filename": filename, "diameter": d})
    summary_df = pd.DataFrame(summary_list)
    summary_df.to_csv(output_csv, index=False)
    pd.DataFrame(detailed_list).to_csv("fiber_diameters_all.csv", index=False)
    print(f"\n✅ Análisis completo. Resultados guardados en '{output_csv}' y 'fiber_diameters_all.csv'.")

def plot_summary_visualizations(summary_csv="fiber_diameters_summary.csv", 
                               detailed_csv="fiber_diameters_all.csv",
                               bin_size=2):
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
    
    summary_df = pd.read_csv(summary_csv)
    detailed_df = pd.read_csv(detailed_csv)
    summary_df = summary_df.sort_values("filename")
    
    
    x = np.arange(len(summary_df))
    labels = summary_df["filename"]

    #Con lo anterior sale como etiquetas los nombres de los archivos, pero si son muy largos se ven mal.
    labels = [f"Frame {i+1}" for i in range(len(summary_df))]

    
    # Gráfico estadísticos (línea + puntos)
    plt.figure(figsize=(14, 7))
    plt.plot(x, summary_df["mean_diameter"], '-o', label="Media", markersize=5)
    plt.plot(x, summary_df["median_diameter"], '-o', label="Mediana", markersize=5)
    plt.plot(x, summary_df["mode_diameter"], '-o', label="Moda", markersize=5)
    plt.plot(x, summary_df["std_diameter"], '-o', label="Desv. estándar", markersize=5)
    plt.xticks(x, labels, rotation=45, ha="right")
    plt.xlabel("Frame (Archivo)")
    plt.ylabel("Diámetro (px)")
    plt.title("Evolución de estadísticos de diámetro de fibras por frame")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Gráfico separado cantidad de fibras (línea + puntos)
    plt.figure(figsize=(14, 4))
    plt.plot(x, summary_df["fiber_count"], '-o', color='gray', label="Cantidad de fibras detectadas", markersize=5)
    plt.xticks(x, labels, rotation=45, ha="right")
    plt.xlabel("Frame (Archivo)")
    plt.ylabel("Cantidad de fibras")
    plt.title("Cantidad de fibras detectadas por frame")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Boxplot por frame
    plt.figure(figsize=(12, 6))
    data_box = [detailed_df[detailed_df["filename"]==fn]["diameter"].values for fn in summary_df["filename"]]
    plt.boxplot(data_box, labels=summary_df["filename"], showfliers=False)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Diámetro (px)")
    plt.title("Boxplot de diámetros de fibras por frame")
    plt.tight_layout()
    plt.show()
    
    # Histograma global con bins más finos
    plt.figure(figsize=(10, 5))
    bins = np.arange(detailed_df["diameter"].min(), detailed_df["diameter"].max() + bin_size, bin_size)
    plt.hist(detailed_df["diameter"], bins=bins, color='skyblue', edgecolor='black')
    plt.xlabel("Diámetro (px)")
    plt.ylabel("Cantidad de fibras")
    plt.title("Histograma global de diámetros de fibras")
    plt.show()


if __name__ == "__main__":
    carpeta_imagenes = r"C:\\Users\\danwo\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au"
    analyze_fiber_diameters(carpeta_imagenes)
    plot_summary_visualizations()
